# Libraries

In [ ]:
!pip install simpledbf
from google.colab import drive
from scipy.interpolate import interp2d
from simpledbf import Dbf5
import pandas as pd
import datetime
import time
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
import time

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for simpledbf: filename=simpledbf-0.2.6-py3-none-any.whl size=13801 sha256=3f4c26eae09bbaba8ccf81fcf35d7241d0660689fb4ac33f681f33f71e555d3e
  Stored in directory: /root/.cache/pip/wheels/7c/4a/85/8d25e6dc4d2968a93a5459ed0153fd6f67041cfece137ca9ce
Successfully built simpledbf


In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


# Open the dataframe

In [ ]:
# Start of the timer
startTime = time.time()

In [ ]:
path = '/content/drive/MyDrive/Colab Notebooks/Agrimonia/Bovini/Bovini.dbf'
df = Dbf5(path).to_dataframe()
df['DATE'] = pd.to_datetime(df['DATE'], format='%d-%b-%Y')
df.rename(columns = {'DATE':'Date'}, inplace = True)
print(type(df))

#Calculate the density
operation = df['NUMEROCAPI']/df['Shape_Area']
df['NUMEROCAPI'] = operation

df.rename(columns = {'NUMEROCAPI':'DENSITY',}, inplace = True)

df.head()

<class 'pandas.core.frame.DataFrame'>


,COD_RIP,COD_REG,COD_PROV,COD_CM,COD_UTS,PRO_COM,PRO_COM_T,COMUNE,COMUNE_A,CC_UTS,Shape_Area,Shape_Leng,CenterX,CenterY,TYPEINT,DENSITY,NALLEVAMENT,BOVINI,BUFALINI,Date
0,1,1,1,201,201,1077,001077,Chiaverano,NaN,0,1.202212e+07,18164.236621,414253.041314,5.039521e+06,0,0.000005,9,62,0,2015-12-31
1,1,1,1,201,201,1079,001079,Chiesanuova,NaN,0,4.118911e+06,10777.318814,394197.394664,5.030811e+06,0,0.000066,7,272,0,2015-12-31
2,1,1,1,201,201,1089,001089,Coazze,NaN,0,5.657268e+07,41591.122092,360808.802237,4.990104e+06,0,0.000001,5,84,0,2015-12-31
3,1,1,1,201,201,1006,001006,Almese,NaN,0,1.787564e+07,17058.439037,375721.501086,4.997294e+06,0,0.000013,21,226,0,2015-12-31
4,1,1,1,201,201,1007,001007,Alpette,NaN,0,5.626076e+06,9795.562269,388609.910901,5.029096e+06,0,0.000000,0,0,0,2015-12-31


In [ ]:
#count rows
count = df.shape
print(count)

(102752, 20)


# Extract informations

In [ ]:
temp = df.loc[:,['PRO_COM']]
temp.head()

,PRO_COM
0,1077
1,1079
2,1089
3,1006
4,1007


In [ ]:
list = temp.PRO_COM.unique()
count_unique = df['PRO_COM'].nunique()  

print(list)
print(type(list))
print("Number of unique cities: ", count_unique)

[ 1077  1079  1089 ... 15251 22254 97008]
<class 'numpy.ndarray'>
Number of unique cities:  7904


# Select the values

In [ ]:
table_to_examine = df.loc[df['PRO_COM'] == 1077]
table_to_examine.head()


,COD_RIP,COD_REG,COD_PROV,COD_CM,COD_UTS,PRO_COM,PRO_COM_T,COMUNE,COMUNE_A,CC_UTS,Shape_Area,Shape_Leng,CenterX,CenterY,TYPEINT,DENSITY,NALLEVAMENT,BOVINI,BUFALINI,Date
0,1,1,1,201,201,1077,001077,Chiaverano,NaN,0,1.202212e+07,18164.236621,414253.041314,5.039521e+06,0,0.000005,9,62,0,2015-12-31
7904,1,1,1,201,201,1077,001077,Chiaverano,NaN,0,1.202212e+07,18164.236621,414253.041314,5.039521e+06,0,0.000005,8,60,0,2016-06-30
15808,1,1,1,201,201,1077,001077,Chiaverano,NaN,0,1.202212e+07,18164.236621,414253.041314,5.039521e+06,0,0.000005,8,66,0,2016-12-31
23712,1,1,1,201,201,1077,001077,Chiaverano,NaN,0,1.202212e+07,18164.236621,414253.041314,5.039521e+06,0,0.000005,8,61,0,2017-06-30
31616,1,1,1,201,201,1077,001077,Chiaverano,NaN,0,1.202212e+07,18164.236621,414253.041314,5.039521e+06,0,0.000005,8,58,0,2017-12-31


## Extract informations

## Functions

### Missing rows

In [ ]:

def fill_missing_rows(df, column, fillwith):
    '''
    Fills the rows of a specific column with a provided list of values.
    Parameters:
        df       (DataFrame)  : The DataFrame to be filled
        column      (string)  : The target column
        fillwith      (list)  : The list of values to fill the column with
    
    Returns:
        df       (DataFrame)  : The filled DataFrame
    '''
    df = df.set_index(column).reindex(pd.Index(fillwith))

    df.reset_index(inplace=True)

    df = df.rename(columns={'index': column})

    return df

### Mising dates

In [ ]:
def fill_missing_dates(df, start, end, freq='d', column='Date'):    
    '''
    Fills the rows of a specific column with the dates between start and end and with a given frequency.
    Parameters:
        df      (DataFrame) : The DataFrame to be filled
        column     (string) : The target column
        start      (string) : The time interval start date
        end        (string) : The time interval end date
        freq       (string) : The frequency of the date ranging
    
    Returns:
        df      (DataFrame) : The filled DataFrame 
    '''
    index = pd.Index(pd.date_range(start, end, freq=freq))

    if len(df) < len(index):

        df[column] = pd.to_datetime(df[column])

        df = fill_missing_rows(df, column, index)

        assert(len(df) == len(index))

    return df

### Interpolation

In [ ]:
def upsample(df, column, start, end, fillna=False):
    '''
    Upsamples the time series by augmenting the DataFrame with the missing dates and interpolating them.
    Parameters:
        df    (DataFrame) : The DataFrame containing the time series to be upsampled
        column   (string) : The column name of the DataFrame where the time series is located
        start    (string) : The time interval start date
        end      (string) : The time interval end date
        fillna  (boolean) : If True fills the remaining missing values
    Returns:
        filled_df (DataFrame) : The DataFrame with the upsampled time series at the specified column
    '''
    filled_df = fill_missing_dates(df, start, end, freq='d', column='Date')
    filled_df[column] = filled_df[column].interpolate()

    if fillna: filled_df[column] = filled_df[column].fillna(method='ffill').fillna(method='bfill')

    return filled_df


In [ ]:
def interpolation(table_to_examine):
  line = table_to_examine.loc[0:1,'COD_RIP':'TYPEINT']
  resultCapi = upsample(table_to_examine, 'DENSITY', '2015-12-31', '2021-12-31')
  resultBovini = upsample(table_to_examine, 'BOVINI', '2015-12-31', '2021-12-31')
  resultBufalini = upsample(table_to_examine, 'BUFALINI', '2015-12-31', '2021-12-31')

  resultCapi.drop(resultCapi.iloc[:,17:20], axis=1, inplace=True)
  resultCapi.drop(resultCapi.iloc[:,1:16], axis=1, inplace=True)
  resultBovini.drop(resultBovini.iloc[:,1:18], axis=1, inplace=True)
  resultBufalini.drop(resultBufalini.iloc[:,1:19], axis=1, inplace=True)
  temp = table_to_examine.drop(table_to_examine.iloc[:,16:19], axis=1, inplace=True)
  template = table_to_examine.iloc[0:1,0:15]
  n = resultBufalini.shape[0]
  df_repeated = pd.concat([template]*n, ignore_index=True)
  df_repeated.insert(0,"Date",resultCapi.loc[:, 'Date'],True)
  df_repeated.insert(16,"DENSITY",resultCapi.loc[:, 'DENSITY'],True) 
  df_repeated.insert(17,"BOVINI",resultBovini.loc[:, 'BOVINI'],True) 
  df_repeated.insert(18,"BUFALINI",resultBufalini.loc[:, 'BUFALINI'],True) 
  return df_repeated

## Call functions

In [ ]:
temp

,PRO_COM
0,1077
1,1079
2,1089
3,1006
4,1007
...,...
102747,26096
102748,22167
102749,15251
102750,22254


In [ ]:
count = 0
result_df  = pd.DataFrame()

for elem in list:
  count+=1
  if((count %100)==0):
    print(" "+str(count))
  print("*", end = '')
  table_to_examine = df.loc[df['PRO_COM'] == elem]
  temp = interpolation(table_to_examine)
  result_df = pd.concat([result_df, temp])

*

<ipython-input-10-d8c1033fdc5d>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[column] = pd.to_datetime(df[column])
/usr/local/lib/python3.8/dist-packages/pandas/core/frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


************************************************************************************************** 100
**************************************************************************************************** 200
**************************************************************************************************** 300
**************************************************************************************************** 400
**************************************************************************************************** 500
**************************************************************************************************** 600
**************************************************************************************************** 700
**************************************************************************************************** 800
**************************************************************************************************** 900
*********************************************************

In [ ]:
print(type(result_df))
print(result_df.shape)

<class 'pandas.core.frame.DataFrame'>
(17333472, 19)


In [ ]:
print(result_df.shape)
result_df.head()

(17333472, 19)


,Date,COD_RIP,COD_REG,COD_PROV,COD_CM,COD_UTS,PRO_COM,PRO_COM_T,COMUNE,COMUNE_A,CC_UTS,Shape_Area,Shape_Leng,CenterX,CenterY,TYPEINT,DENSITY,BOVINI,BUFALINI
0,2015-12-31,1,1,1,201,201,1077,001077,Chiaverano,NaN,0,1.202212e+07,18164.236621,414253.041314,5.039521e+06,0,0.000005,62.000000,0.0
1,2016-01-01,1,1,1,201,201,1077,001077,Chiaverano,NaN,0,1.202212e+07,18164.236621,414253.041314,5.039521e+06,0,0.000005,61.989011,0.0
2,2016-01-02,1,1,1,201,201,1077,001077,Chiaverano,NaN,0,1.202212e+07,18164.236621,414253.041314,5.039521e+06,0,0.000005,61.978022,0.0
3,2016-01-03,1,1,1,201,201,1077,001077,Chiaverano,NaN,0,1.202212e+07,18164.236621,414253.041314,5.039521e+06,0,0.000005,61.967033,0.0
4,2016-01-04,1,1,1,201,201,1077,001077,Chiaverano,NaN,0,1.202212e+07,18164.236621,414253.041314,5.039521e+06,0,0.000005,61.956044,0.0


# Save the result

In [ ]:
result_df.to_csv('/content/drive/MyDrive/Colab Notebooks/Agrimonia/Bovini/resultBoviniAgrimonia.csv')

## Time

In [ ]:
executionTime = (time.time() - startTime)/60
print('Execution time in minutes: ' + str(executionTime))

Execution time in seconds: 161.5222236951192
